In [76]:
import pandas as pd
ddf = pd.read_csv("traing.csv")
ddf

,Language,Code,CPU_Time(ms),Memory(KB),Energy(kWh),Carbon_Emission(gCO₂),Efficiency_Label
0,Python,def nested_loop(arr):\n for i in arr:\n ...,618,8057,0.017167,6.866667,1
1,Python,def bubble_sort(arr):\n n = len(arr)\n f...,946,7896,0.026278,10.511111,1
2,Python,def merge_sort(arr):\n if len(arr) > 1:\n ...,877,5945,0.024361,9.744444,0
3,Python,def find_max(arr):\n max_val = arr[0]\n ...,756,3138,0.021000,8.400000,0
4,Python,def nested_loop(arr):\n for i in arr:\n ...,505,4604,0.014028,5.611111,1
...,...,...,...,...,...,...,...
9995,JavaScript,function bubbleSort(arr) {\n let n = arr.leng...,824,5136,0.022889,9.155556,1
9996,JavaScript,function sumArray(arr) {\n return arr.reduce(...,952,6098,0.026444,10.577778,0
9997,JavaScript,function nestedLoop(arr) {\n arr.forEach(i =>...,262,5473,0.007278,2.911111,1
9998,JavaScript,function quickSort(arr) {\n if (arr.length <=...,835,8148,0.023194,9.277778,0


In [11]:
  import ast

  def analyze_python_code(code):
      tree = ast.parse(code)
      functions = []
      loops = []
      variables = set()
      control_flows = []

      for node in ast.walk(tree):
          if isinstance(node, ast.FunctionDef):
              functions.append(node.name)
          elif isinstance(node, (ast.For, ast.While)):
              loops.append(node)
          elif isinstance(node, ast.Name):
              variables.add(node.id)
          elif isinstance(node, (ast.If, ast.Try)):
              control_flows.append(node)

      return {
          'functions': functions,
          'loops': len(loops),
          'variables': list(variables),
          'control_flows': len(control_flows)
      }



In [13]:
import re
import pandas as pd

def analyze_javascript_code(code):
    """
    Analyze JavaScript code to extract functions, loops, variables, and control flows
    using regular expressions without any external dependencies.
    """
    try:
        # Find function declarations like "function name() {}"
        function_decls = re.findall(r'function\s+(\w+)', code)
        
        # Find function expressions like "const name = function() {}" or "const name = () => {}"
        func_expressions = re.findall(r'(?:var|let|const)\s+(\w+)\s*=\s*function', code)
        arrow_funcs = re.findall(r'(?:var|let|const)\s+(\w+)\s*=\s*(?:\([^)]*\)|[^=]*)\s*=>', code)
        
        # Combine all functions
        functions = function_decls + func_expressions + arrow_funcs
        
        # Anonymous arrow functions that we can't easily get the name of
        anonymous_arrows = len(re.findall(r'=>', code)) - len(arrow_funcs)
        for i in range(anonymous_arrows):
            functions.append(f"anonymous_arrow_{i}")
        
        # Count loops: for, while, do-while
        for_loops = len(re.findall(r'\bfor\s*\(', code))
        while_loops = len(re.findall(r'\bwhile\s*\(', code))
        do_while_loops = len(re.findall(r'\bdo\s*\{', code))
        loops = for_loops + while_loops + do_while_loops
        
        # Find variable declarations
        var_declarations = re.findall(r'\b(?:var|let|const)\s+(\w+)', code)
        
        # Count control flow statements
        if_statements = len(re.findall(r'\bif\s*\(', code))
        switch_statements = len(re.findall(r'\bswitch\s*\(', code))
        try_statements = len(re.findall(r'\btry\s*\{', code))
        ternary_operators = len(re.findall(r'\?\s*[^:]+\s*:', code))
        control_flows = if_statements + switch_statements + try_statements + ternary_operators
        
        return {
            'functions': functions,
            'loops': loops,
            'variables': var_declarations,
            'control_flows': control_flows
        }
    except Exception as e:
        return {
            'error': str(e),
            'functions': [],
            'loops': 0,
            'variables': [],
            'control_flows': 0
        }




In [15]:
  import tokenize
  from io import BytesIO

  def tokenize_python_code(code):
      tokens = []
      reader = BytesIO(code.encode('utf-8')).readline
      for tok in tokenize.tokenize(reader):
          tokens.append((tokenize.tok_name[tok.type], tok.string))
      return tokens



In [17]:
import re
from io import StringIO
import pandas as pd

def tokenize_javascript_code(code):
    """
    A JavaScript tokenizer implemented in Python that provides
    output similar to Python's tokenize module.
    Returns a list of (token_type, token_value) tuples.
    """
    # Start with encoding token
    tokens = [('ENCODING', 'utf-8')]
    
    # Define patterns for different token types
    patterns = [
        # Comments
        (r'\/\/.*?$|\/\*[\s\S]*?\*\/', 'COMMENT'),
        
        # Strings
        (r'"(?:\\.|[^"\\])*"', 'STRING'),
        (r"'(?:\\.|[^'\\])*'", 'STRING'),
        (r"`(?:\\.|[^`\\])*`", 'STRING'),
        
        # Numbers
        (r'\b\d+(?:\.\d+)?(?:[eE][+-]?\d+)?\b', 'NUMBER'),
        
        # Keywords
        (r'\b(var|let|const|function|if|else|for|while|do|switch|case|break|continue|return|new|try|catch|finally|throw|class|this|super|import|export|from|as|async|await|yield|delete|typeof|instanceof|in|of|void|default|extends|implements|interface|package|private|protected|public|static)\b', 'KEYWORD'),
        
        # Operators
        (r'=>|===|!==|==|!=|>=|<=|\+\+|--|&&|\|\||\+=|-=|\*=|/=|%=|\*\*=|<<=|>>=|>>>=|&=|\|=|\^=', 'OP'),
        (r'[+\-*/%&|^~<>=!?:]', 'OP'),
        
        # Separators
        (r'[\[\](){}]', 'OP'),
        (r';', 'OP'),
        (r',', 'OP'),
        (r'\.', 'OP'),
        
        # Identifiers (must come after keywords)
        (r'[a-zA-Z_$][a-zA-Z0-9_$]*', 'NAME'),
        
        # Newlines
        (r'\n', 'NL'),
        
        # Whitespace
        (r'[ \t\r]+', 'WHITESPACE'),
    ]
    
    # Function to find the next token
    def find_next_token(text, pos):
        for pattern, token_type in patterns:
            regex = re.compile(pattern)
            match = regex.match(text, pos)
            if match:
                return token_type, match.group(0), match.end()
        
        # If no pattern matches, return just the current character
        if pos < len(text):
            return 'OP', text[pos], pos + 1
        return None, None, pos
    
    # Process the code character by character
    pos = 0
    while pos < len(code):
        token_type, token_value, new_pos = find_next_token(code, pos)
        if token_type:
            # Skip whitespace tokens but keep newlines
            if token_type != 'WHITESPACE':
                # Convert keywords to NAME to match Python's tokenizer behavior
                if token_type == 'KEYWORD':
                    token_type = 'NAME'
                tokens.append((token_type, token_value))
            pos = new_pos
        else:
            break
    
    # Add the ENDMARKER token
    tokens.append(('NEWLINE', ''))
    tokens.append(('ENDMARKER', ''))
    
    return tokens

# Example usage function
def tokenize_code(df):
    lang = df.iloc[9, 0]
    code = df.iloc[0, 1]
    
    if lang == "JavaScript":
        tokens = tokenize_javascript_code(code)
        return tokens
    elif lang == "Python":
        import tokenize
        from io import BytesIO
        
        def tokenize_python_code(code):
            tokens = []
            reader = BytesIO(code.encode('utf-8')).readline
            for tok in tokenize.tokenize(reader):
                tokens.append((tokenize.tok_name[tok.type], tok.string))
            return tokens
        
        tokens = tokenize_python_code(code)
        return tokens
    else:
        return [("ERROR", f"Unsupported language: {lang}")]

 


In [78]:
import pandas as pd

data = {
    'Language': [],
    'Code':[]
}
for index, row in ddf.iterrows():
    data['Language'].append(row['Language'])
    data['Code'].append(row['Code'])

    df = pd.DataFrame(data)

# Function to analyze code
    def analyze_code(row):
        if row['Language'] == 'Python':
            analysis = analyze_python_code(row['Code'])
            tokens = tokenize_python_code(row['Code'])
        elif row['Language'] == 'JavaScript':
            analysis = analyze_javascript_code(row['Code'])
            tokens = tokenize_javascript_code(row['Code'])
        else:
            analysis = {}
            tokens = []
    
        return pd.Series({
            'Functions': analysis.get('functions', []),
            'Loops': analysis.get('loops', 0),
            'Variables': analysis.get('variables', []),
            'Control_Flows': analysis.get('control_flows', 0),
            'Tokens': tokens
        })


df = df.join(df.apply(analyze_code, axis=1))






,Language,Code,Functions,Loops,Variables,Control_Flows,Tokens
0,Python,def nested_loop(arr):\n for i in arr:\n ...,[nested_loop],2,"[print, arr, i, j]",0,"[(ENCODING, utf-8), (NAME, def), (NAME, nested..."
1,Python,def bubble_sort(arr):\n n = len(arr)\n f...,[bubble_sort],2,"[n, i, len, range, arr, j]",1,"[(ENCODING, utf-8), (NAME, def), (NAME, bubble..."
2,Python,def merge_sort(arr):\n if len(arr) > 1:\n ...,[merge_sort],0,"[len, merge_sort, arr, L, R, mid]",1,"[(ENCODING, utf-8), (NAME, def), (NAME, merge_..."
3,Python,def find_max(arr):\n max_val = arr[0]\n ...,[find_max],1,"[max_val, arr, i]",1,"[(ENCODING, utf-8), (NAME, def), (NAME, find_m..."
4,Python,def nested_loop(arr):\n for i in arr:\n ...,[nested_loop],2,"[print, arr, i, j]",0,"[(ENCODING, utf-8), (NAME, def), (NAME, nested..."
...,...,...,...,...,...,...,...
9995,JavaScript,function bubbleSort(arr) {\n let n = arr.leng...,[bubbleSort],2,"[n, i, j]",1,"[(ENCODING, utf-8), (NAME, function), (NAME, b..."
9996,JavaScript,function sumArray(arr) {\n return arr.reduce(...,"[sumArray, anonymous_arrow_0]",0,[],0,"[(ENCODING, utf-8), (NAME, function), (NAME, s..."
9997,JavaScript,function nestedLoop(arr) {\n arr.forEach(i =>...,"[nestedLoop, anonymous_arrow_0, anonymous_arro...",0,[],0,"[(ENCODING, utf-8), (NAME, function), (NAME, n..."
9998,JavaScript,function quickSort(arr) {\n if (arr.length <=...,"[quickSort, left, right]",0,"[pivot, left, right]",1,"[(ENCODING, utf-8), (NAME, function), (NAME, q..."


In [86]:
new_df = pd.concat([ddf, df[['Functions', 'Loops', 'Variables', 'Control_Flows', 'Tokens']]], axis=1)

In [104]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Create an instance of OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
encoded_column = encoder.fit_transform(new_df[['Language']])

# Create a new DataFrame with the encoded values
encoded_df = pd.DataFrame(encoded_column, columns=encoder.get_feature_names_out(['Language']))

# Concatenate with the original DataFrame
new_df = pd.concat([new_df, encoded_df], axis=1)


In [112]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# List of features to normalize
features_to_normalize = ['CPU_Time(ms)', 'Memory(KB)', 'Energy(kWh)', 'Carbon_Emission(gCO₂)']

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit and transform the selected features
new_df[features_to_normalize] = scaler.fit_transform(new_df[features_to_normalize])


In [122]:
new_df.to_csv('filename.csv', index=False)


In [120]:
import gym
import numpy as np
import pandas as pd

class CodeEfficiencyEnv(gym.Env):
    def __init__(self, df):
        super(CodeEfficiencyEnv, self).__init__()
        self.df = df
        self.index = 0
        
        # Define Action Space (0: Keep Code, 1: Suggest Optimization)
        self.action_space = gym.spaces.Discrete(2)

        # Define State Space (CPU_Time, Memory, Energy, Carbon_Emission, Loops, Control_Flows)
        self.observation_space = gym.spaces.Box(low=0, high=100000, shape=(6,), dtype=np.float32)

    def get_state(self):
        row = self.df.iloc[self.index]
        return np.array([row['CPU_Time(ms)'], row['Memory(KB)'], row['Energy(kWh)'],
                         row['Carbon_Emission(gCO₂)'], row['Loops'], row['Control_Flows']], dtype=np.float32)

    def step(self, action):
        reward = 0
        
        # If action = Efficiency_Label => Reward +1
        # If action != Efficiency_Label => Reward -1
        if action == self.df.iloc[self.index]['Efficiency_Label']:
            reward = 1
        else:
            reward = -1

        self.index += 1
        
        # Done when all rows are processed
        done = self.index >= len(self.df)
        
        if not done:
            state = self.get_state()
        else:
            state = None
        
        return state, reward, done, {}

    def reset(self):
        self.index = 0
        return self.get_state()


SyntaxError: invalid syntax (2202935506.py, line 8)

In [118]:
env = CodeEfficiencyEnv(df)
state = env.reset()
done = False

while not done:
    action = np.random.choice([0, 1])  # Random action
    state, reward, done, _ = env.step(action)
    print(f"Action: {action}, Reward: {reward}")


KeyError: 'CPU_Time(ms)'